In [1]:
import os
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import numpy as np
from sklearn.metrics import f1_score,accuracy_score,confusion_matrix
from collections import defaultdict
import pickle
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

### General Functions 

In [2]:
# Unpickle Function for the CIFAR-10 dataset
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

#save the model
def save_model(model):
    filename = 'SVM_model.sav'
    pickle.dump(model, open(filename, 'wb'))
    
def get_model():
    if os.path.isfile('SVM_model.sav'):
        svc=None
        flag_was_model_saved=False
        with open('SVM_model.sav', 'rb') as f:
            svc = pickle.load(f)
            flag_was_model_saved=True
        return [svc,flag_was_model_saved]
    
def combineData(data,limit=None):
    combined_data = defaultdict(list)
    for d in data:
        for key, value_list in d.items():
            if limit is not None:
                value_list = value_list[:limit]
            combined_data[key].extend(value_list)
    return combined_data

### Variables and Files

In [3]:
# Load Data
dir='cifar-10-batches-py/'

In [4]:
# Get the files
files=os.listdir(dir)
files=[dir+file for file in files]
files.sort()

In [5]:
# Get the test file
test_file=files[len(files)-1]

# index file and test File are not needed
files=files[:len(files)-2]

In [6]:
# Get the data
data=[unpickle(file) for file in files]
test_file=unpickle(test_file)

data_names=data[0]
data=data[1:]

### Run everything before this for init

### SVC Model

In [8]:

#get the model from file and flag to fit the model
svc,flag_was_model_saved=get_model()

if(not flag_was_model_saved):
    parameters={'kernel':('linear', 'rbf'), 'C':[1, 10]}
    svc=SVC()
    clf=GridSearchCV(svc,parameters)
    

Model ready: GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')})


##### Splitting Data

In [25]:
combined_data=combineData(data)

In [9]:
# Get the data
X_test , y_test = np.array(test_file[b'data']),np.array(test_file[b'labels'])
X_train , y_train = np.array(combined_data[b'data']),np.array(combined_data[b'labels'])

#### save if not and fit the model

In [10]:
# Fit and Predict
if(not flag_was_model_saved):
    svc=clf.fit(X_train, y_train)
    save_model(svc)
y_pred=svc.predict(X_test)

#### Prediction scores

In [11]:
# Get the classification report
print(classification_report(y_test, y_pred))

# Get the accuracy
print('Accuracy: ', accuracy_score(y_test, y_pred))

# Get the confusion matrix
print('Confusion Matrix: ')
print(confusion_matrix(y_test, y_pred))

# Get the F1 score
print('F1 Score: ', f1_score(y_test, y_pred, average='weighted'))

              precision    recall  f1-score   support

           0       0.62      0.67      0.64      1000
           1       0.65      0.67      0.66      1000
           2       0.45      0.47      0.46      1000
           3       0.38      0.41      0.39      1000
           4       0.50      0.48      0.49      1000
           5       0.50      0.47      0.48      1000
           6       0.62      0.60      0.61      1000
           7       0.66      0.59      0.62      1000
           8       0.71      0.69      0.70      1000
           9       0.62      0.63      0.63      1000

    accuracy                           0.57     10000
   macro avg       0.57      0.57      0.57     10000
weighted avg       0.57      0.57      0.57     10000

Accuracy:  0.5687
Confusion Matrix: 
[[667  32  53  27  20  20  18  19  97  47]
 [ 37 670  18  31  13  17   7  23  46 138]
 [ 70  21 474  96 110  60  80  54  16  19]
 [ 25  34  96 408  68 169  87  49  21  43]
 [ 50  13 155  63 484  47  88  6

### First 1000 values of each batch

In [13]:
# Paramater tuning
parameters={'kernel':('linear', 'rbf'), 'C':[1, 10]}


In [14]:
# Initialize the classifier
svc=SVC()
clf=GridSearchCV(svc,parameters)

In [8]:
# Get the data and split them
combined_data=combineData(data,limit=1000)
X_test , y_test = np.array(test_file[b'data']),np.array(test_file[b'labels'])
X_train , y_train = np.array(combined_data[b'data']),np.array(combined_data[b'labels'])

In [16]:
# fit the model
clf.fit(X_train, y_train)

GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')})

In [21]:
clf.best_params_
clf.cv_results_

{'mean_fit_time': array([36.93399696, 33.08270164, 37.47338996, 39.72121539]),
 'std_fit_time': array([1.90118109, 2.08065237, 0.88874958, 2.23688123]),
 'mean_score_time': array([ 5.81222   , 10.79990916,  6.14381971,  9.89539266]),
 'std_score_time': array([0.32724059, 1.22245531, 0.2971546 , 0.42606543]),
 'param_C': masked_array(data=[1, 1, 10, 10],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['linear', 'rbf', 'linear', 'rbf'],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'linear'},
  {'C': 1, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'}],
 'split0_test_score': array([0.335, 0.412, 0.335, 0.434]),
 'split1_test_score': array([0.304, 0.442, 0.304, 0.447]),
 'split2_test_score': array([0.328, 0.426, 0.328, 0.43 ]),
 'split3_test_score': array([0.291, 0.422, 0.291, 0.446]),


### Using PCA 

In [26]:
# pca
# 90% variance
pca=PCA(n_components=.90)
# model filt
pca.fit(X_train)


PCA(n_components=0.5)

In [14]:
# transform the data
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [15]:
# fit the model
svc,_=get_model()
svc.fit(X_train_pca, y_train)

SVC(C=10)

In [16]:
svc.get_params()

{'C': 10,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [25]:
svc.score(X_test_pca,y_test) 
# 90% variance -> 0.4391
# 50% variance -> 0.2809

0.4391

## SVHN

### Declarations

In [2]:
# To get the matlab files into python
import h5py

In [3]:
dir_train = 'SVHN/train/'
dir_test = 'SVHN/test/'

### Functions to extract data from .mat files

In [7]:
# Extract Data from .mat file
def get_names(h5_file):
    """ Get the names from the 'name' dataset """
    names = h5_file['digitStruct']['name']
    name_list = []
    for i in range(names.shape[0]):
        name_ref = names[i][0]
        name = ''.join(chr(c[0]) for c in h5_file[name_ref])
        name_list.append(name)
    return name_list

def get_bbox(h5_file, index):
    """ Get the bounding box data for a given index """
    item = h5_file['digitStruct']['bbox'][index].item()
    bbox = {}
    keys = ['height', 'left', 'top', 'width', 'label']

    for key in keys:
        attribute = h5_file[item][key]
        if attribute.shape[0] == 1:
            bbox[key] = [attribute[0][0]]
        else:
            bbox[key] = [h5_file[attribute[value].item()][()][0][0] for value in range(attribute.shape[0])]
    
    return bbox

### Get information about the images

In [4]:

# Open the HDF5 file
def getImages(dir):
    images=[]
    with h5py.File(dir+'digitStruct.mat', 'r') as h5_file:
        names = get_names(h5_file)

        for i, name in enumerate(names):
            bbox = get_bbox(h5_file, i)
            images.append([name,bbox])
            # print(f"Image: {name}, BBox: {bbox}")
    return images

In [8]:
images_train=getImages(dir_train)
images_test=getImages(dir_test)

### Image processing

In [9]:
from PIL import Image

def crop_digits(image_path, bbox_info):
    # Crop the digits from the image based on the bounding box information
    image = Image.open(image_path)
    digit_images = []

    for i in range(len(bbox_info['label'])):
        left = int(bbox_info['left'][i])
        top = int(bbox_info['top'][i])
        right = left + int(bbox_info['width'][i])
        bottom = top + int(bbox_info['height'][i])

        digit_image = image.crop((left, top, right, bottom))
        digit_images.append(digit_image)

    return digit_images

def getDigitImages(dir,images):
    digit_images=[]
    for image in images:
        digit_images.append(crop_digits(dir+image[0], image[1]))
    return digit_images

In [10]:
digit_images_train=getDigitImages(dir_train,images_train)
digit_images_test=getDigitImages(dir_test,images_test)

### Data Preparation

In [11]:
def preprocess_for_svm(digit_images, size=(32, 32)):
    processed_images = []
    for img in digit_images:
                
        img = img[0].resize(size).convert('L')  # Convert to grayscale and resize
        img_data = np.array(img, dtype='float32').flatten()
        img_data /= 255.0  # Normalize pixel values
        processed_images.append(img_data)

    return np.array(processed_images)

# Preprocess images
X_train = preprocess_for_svm(digit_images_train)
X_test = preprocess_for_svm(digit_images_test)


### SVM classification

#### Because there was more than one labels for each image, i choose the first one in all cases


In [ ]:
# Get the labels for each image
y_test=[]
for list in images_test:
    y_test.append(list[1]['label'][0])
    
y_train=[]
for list in images_train:
    y_train.append(list[1]['label'][0])

In [22]:
# Train the SVM
svc = SVC(C=10,kernel='rbf')  # You can experiment with different kernels
svc.fit(X_train, y_train)

SVC(C=10)

In [23]:
# Evaluate the SVM
accuracy = svc.score(X_test, y_test)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7869605142332415


### Using multi-label targets

In [12]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier

In [13]:
y_test=[]
for list in images_test:
    y_test.append(list[1]['label'])
    
y_train=[]
for list in images_train:
    y_train.append(list[1]['label'])

In [14]:
mlb = MultiLabelBinarizer()
y_train_bin = mlb.fit_transform(y_train)
y_test_bin = mlb.transform(y_test)

In [15]:
svc = OneVsRestClassifier(SVC(kernel='rbf',C=10))

# Train the SVM
svc.fit(X_train, y_train_bin)

# Evaluate the SVM
accuracy = svc.score(X_test, y_test_bin)
print(f"Accuracy: {accuracy}")

Accuracy: 0.13575145393327212
